# Lab 8-2:XOR NN
은닉층이 하나인 신경망은 XOR 문제를 풀 수 없다. 하지만 은닉층이 두 개인 신경망을 이용하면 이 문제를 풀 수 있다.
여기서는 은닉층이 두 개인 신경망으로 XOR 문제를 푸는 방법을 다룬다.

## Imports

In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

## Data

In [2]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

## Model

In [3]:
w1 = torch.Tensor(2, 2).to(device)
b1 = torch.Tensor(2).to(device)

w2 = torch.Tensor(1, 2).to(device)
b2 = torch.Tensor(1).to(device)

# Binary Cross Entropy
def criterion(hypothesis, y):
  return (y * torch.log(hypothesis) + (1 - y) * torch.log(1 - hypothesis)).mean() * (-1)

def sigmoid(x):
  return 1 / (1 + torch.exp(-x))

#derivative of sigmoid
def d_sigmoid(x):
  return sigmoid(x) * (1 - sigmoid(x))

for step in range(1000):
  #forward
  l1 = w1 @ X + b1
  a1 = sigmoid(l1)
  l2 = w2 @ a1 + b2
  prediction = sigmoid(l2)

  cost = criterion(prediction, Y)

  #Backpropagation
  #Cost derivative
  d_prediction = (prediction - Y) / (prediction * (1.0 - prediction) * 1e-7)
  
  #Layer 2
  d_l2 = d_prediction * d_sigmoid(l2)
  d_b2 = d_l2 * 1
  d_w2 = d_l2 * a1
  d_w2o = torch.transpose(a1, 0, 1) @ d_b2

  #Layer 1
  d_a1 = d_l2 * w2